# Solution 

## Gradient of Naive-Softmax Loss with Respect to Center Vector

## (b): Computing the Partial Derivative

### i
We have:
- $J_{\text{naive-softmax}}(\mathbf{v}_c, o, \mathbf{U}) = -\log P(O = o | C = c)$
- $P(O = o | C = c) = \frac{\exp(\mathbf{u}_o^T \mathbf{v}_c)}{\sum_{w=1}^{|V|} \exp(\mathbf{u}_w^T \mathbf{v}_c)} = \hat{y}_o$

#### Step 1: Express the loss in terms of softmax
$$J = -\log \hat{y}_o = -\log \frac{\exp(\mathbf{u}_o^T \mathbf{v}_c)}{\sum_{w=1}^{|V|} \exp(\mathbf{u}_w^T \mathbf{v}_c)}$$

$$= -\mathbf{u}_o^T \mathbf{v}_c + \log \sum_{w=1}^{|V|} \exp(\mathbf{u}_w^T \mathbf{v}_c)$$

#### Step 2: Compute the gradient
$$\frac{\partial J}{\partial \mathbf{v}_c} = \frac{\partial}{\partial \mathbf{v}_c}\left[-\mathbf{u}_o^T \mathbf{v}_c + \log \sum_{w=1}^{|V|} \exp(\mathbf{u}_w^T \mathbf{v}_c)\right]$$

For the first term:
$$\frac{\partial}{\partial \mathbf{v}_c}(-\mathbf{u}_o^T \mathbf{v}_c) = -\mathbf{u}_o$$

For the second term, using chain rule:
$$\frac{\partial}{\partial \mathbf{v}_c}\log \sum_{w=1}^{|V|} \exp(\mathbf{u}_w^T \mathbf{v}_c) = \frac{1}{\sum_{w=1}^{|V|} \exp(\mathbf{u}_w^T \mathbf{v}_c)} \cdot \sum_{w=1}^{|V|} \exp(\mathbf{u}_w^T \mathbf{v}_c) \mathbf{u}_w$$

$$= \frac{\sum_{w=1}^{|V|} \exp(\mathbf{u}_w^T \mathbf{v}_c) \mathbf{u}_w}{\sum_{w=1}^{|V|} \exp(\mathbf{u}_w^T \mathbf{v}_c)} = \sum_{w=1}^{|V|} \frac{\exp(\mathbf{u}_w^T \mathbf{v}_c)}{\sum_{k=1}^{|V|} \exp(\mathbf{u}_k^T \mathbf{v}_c)} \mathbf{u}_w$$

$$= \sum_{w=1}^{|V|} \hat{y}_w \mathbf{u}_w$$

#### Step 3: Combine terms
$$\frac{\partial J}{\partial \mathbf{v}_c} = -\mathbf{u}_o + \sum_{w=1}^{|V|} \hat{y}_w \mathbf{u}_w$$

#### Step 4: Express in vectorized form using 𝐲, 𝐲̂, and 𝐔

Note that:
- $\mathbf{u}_o = \mathbf{U} \mathbf{y}$ (since $\mathbf{y}$ is one-hot with $y_o = 1$)
- $\sum_{w=1}^{|V|} \hat{y}_w \mathbf{u}_w = \mathbf{U} \hat{\mathbf{y}}$

Therefore:
$$\boxed{\frac{\partial J_{\text{naive-softmax}}}{\partial \mathbf{v}_c} = \mathbf{U}(\hat{\mathbf{y}} - \mathbf{y})}$$


### ii When is the gradient equal to zero?

The gradient equals zero when:
$$\mathbf{U}(\hat{\mathbf{y}} - \mathbf{y}) = \mathbf{0}$$

This occurs when $\hat{\mathbf{y}} - \mathbf{y} = \mathbf{0}$, i.e., when $\hat{\mathbf{y}} = \mathbf{y}$.

**Answer**: The gradient is zero when the predicted probability distribution perfectly matches the true distribution (when $\hat{y}_o = 1$ and $\hat{y}_w = 0$ for all $w \neq o$).

#### Interpretation of the gradient terms

The gradient can be written as:
$$\frac{\partial J}{\partial \mathbf{v}_c} = \mathbf{U}\hat{\mathbf{y}} - \mathbf{U}\mathbf{y}$$

#### Term 1: $\mathbf{U}\hat{\mathbf{y}}$ (Predicted Context)
- This is a weighted average of all word vectors, weighted by their predicted probabilities
- It represents the "expected" context vector based on the current model's predictions
- When subtracted from $\mathbf{v}_c$, it pulls the center vector away from irrelevant word directions

#### Term 2: $-\mathbf{U}\mathbf{y}$ (True Context)
- This is simply $-\mathbf{u}_o$, the negative of the true output word vector
- When subtracted from $\mathbf{v}_c$, the double negative means we add $\mathbf{u}_o$
- This pushes the center vector toward the true output word vector

#### Combined Effect
When we update $\mathbf{v}_c := \mathbf{v}_c - \alpha \frac{\partial J}{\partial \mathbf{v}_c}$:

1. **Attraction**: The center vector is pulled toward the true output word vector ($\mathbf{u}_o$)
2. **Repulsion**: The center vector is pushed away from the weighted average of all word vectors (especially those with high predicted probability but shouldn't be there)

This creates the desired effect: making the center vector more similar to the true context word and less similar to incorrect words that the model incorrectly predicts with high probability.

## (c): When L2 Normalization Takes Away Useful Information

L2 normalization removes useful information when the **magnitude** of word embeddings carries semantic meaning that's relevant to the classification task.

Consider the hint: if $\mathbf{u}_x = \alpha\mathbf{u}_y$ where $\alpha > 0$, then after L2 normalization:
- $\frac{\mathbf{u}_x}{||\mathbf{u}_x||_2} = \frac{\alpha\mathbf{u}_y}{||\alpha\mathbf{u}_y||_2} = \frac{\alpha\mathbf{u}_y}{|\alpha|||\mathbf{u}_y||_2} = \frac{\mathbf{u}_y}{||\mathbf{u}_y||_2}$

So normalized $\mathbf{u}_x$ and normalized $\mathbf{u}_y$ become **identical**.

This is problematic when words have the same semantic direction but different **intensity**. For example:
- "good" vs "excellent": If $\mathbf{u}_{\text{excellent}} = 3\mathbf{u}_{\text{good}}$, the magnitude difference captures that "excellent" is more positive than "good"
- "bad" vs "terrible": Similarly, if $\mathbf{u}_{\text{terrible}} = 2\mathbf{u}_{\text{bad}}$, the magnitude indicates stronger negativity

In the classification task, the phrase "This movie is excellent" should contribute more positive signal than "This movie is good". But after normalization, both words contribute equally, losing the intensity information encoded in the original magnitudes.

### When L2 Normalization Doesn't Take Away Useful Information

L2 normalization preserves useful information when only the **direction** (semantic meaning) matters, not the magnitude.

This occurs when:

1. **Magnitude differences are noise**: If embedding magnitudes reflect training artifacts, word frequency, or other non-semantic factors rather than semantic intensity, normalization removes this noise.

2. **Words are semantically equivalent**: When $\mathbf{u}_x = \alpha\mathbf{u}_y$ represents true semantic equivalence (like synonyms "happy" and "joyful"), the magnitude difference might be spurious, and normalization correctly treats them equally.

3. **Downstream task is direction-sensitive only**: If the classification boundary depends only on the overall semantic direction of the phrase embedding (sum of individual embeddings), not its magnitude, then normalization can actually improve performance by removing irrelevant scale variations.

### Summary

- **Normalization hurts** when magnitude encodes semantic intensity (degree of positivity/negativity)
- **Normalization helps** when magnitude represents noise or when only semantic direction matters for the task

The key insight is that L2 normalization fundamentally changes the contribution weighting scheme from magnitude-dependent to purely directional, which may or may not align with the semantic structure relevant to your downstream task.

## (d): Computing the partial derivatives

The loss can be written as:
$$J = -\log \hat{y}_o = -\log P(O=o|C=c)$$

Taking the partial derivative with respect to $\mathbf{u}_w$:

$$\frac{\partial J}{\partial \mathbf{u}_w} = -\frac{1}{\hat{y}_o} \frac{\partial \hat{y}_o}{\partial \mathbf{u}_w}$$

Now I need to compute $\frac{\partial \hat{y}_o}{\partial \mathbf{u}_w}$.

Since $\hat{y}_o = \frac{\exp(\mathbf{u}_o^T \mathbf{v}_c)}{\sum_{k} \exp(\mathbf{u}_k^T \mathbf{v}_c)}$, using the quotient rule:

$$\frac{\partial \hat{y}_o}{\partial \mathbf{u}_w} = \frac{\frac{\partial}{\partial \mathbf{u}_w}[\exp(\mathbf{u}_o^T \mathbf{v}_c)] \cdot \sum_{k} \exp(\mathbf{u}_k^T \mathbf{v}_c) - \exp(\mathbf{u}_o^T \mathbf{v}_c) \cdot \frac{\partial}{\partial \mathbf{u}_w}[\sum_{k} \exp(\mathbf{u}_k^T \mathbf{v}_c)]}{[\sum_{k} \exp(\mathbf{u}_k^T \mathbf{v}_c)]^2}$$

### Case 1: When $w = o$

$$\frac{\partial}{\partial \mathbf{u}_o}[\exp(\mathbf{u}_o^T \mathbf{v}_c)] = \exp(\mathbf{u}_o^T \mathbf{v}_c) \mathbf{v}_c$$

$$\frac{\partial}{\partial \mathbf{u}_o}[\sum_{k} \exp(\mathbf{u}_k^T \mathbf{v}_c)] = \exp(\mathbf{u}_o^T \mathbf{v}_c) \mathbf{v}_c$$

Therefore:
$$\frac{\partial \hat{y}_o}{\partial \mathbf{u}_o} = \frac{\exp(\mathbf{u}_o^T \mathbf{v}_c) \mathbf{v}_c \cdot \sum_{k} \exp(\mathbf{u}_k^T \mathbf{v}_c) - \exp(\mathbf{u}_o^T \mathbf{v}_c) \cdot \exp(\mathbf{u}_o^T \mathbf{v}_c) \mathbf{v}_c}{[\sum_{k} \exp(\mathbf{u}_k^T \mathbf{v}_c)]^2}$$

$$= \frac{\exp(\mathbf{u}_o^T \mathbf{v}_c) \mathbf{v}_c [\sum_{k} \exp(\mathbf{u}_k^T \mathbf{v}_c) - \exp(\mathbf{u}_o^T \mathbf{v}_c)]}{[\sum_{k} \exp(\mathbf{u}_k^T \mathbf{v}_c)]^2}$$

$$= \hat{y}_o \mathbf{v}_c (1 - \hat{y}_o)$$

So: $$\frac{\partial J}{\partial \mathbf{u}_o} = -\frac{1}{\hat{y}_o} \cdot \hat{y}_o \mathbf{v}_c (1 - \hat{y}_o) = -\mathbf{v}_c (1 - \hat{y}_o) = \mathbf{v}_c(\hat{y}_o - 1)$$

Since $y_o = 1$:
$$\boxed{\frac{\partial J}{\partial \mathbf{u}_o} = \mathbf{v}_c(\hat{y}_o - y_o)}$$

### Case 2: When $w \neq o$

$$\frac{\partial}{\partial \mathbf{u}_w}[\exp(\mathbf{u}_o^T \mathbf{v}_c)] = 0$$

$$\frac{\partial}{\partial \mathbf{u}_w}[\sum_{k} \exp(\mathbf{u}_k^T \mathbf{v}_c)] = \exp(\mathbf{u}_w^T \mathbf{v}_c) \mathbf{v}_c$$

Therefore:
$$\frac{\partial \hat{y}_o}{\partial \mathbf{u}_w} = \frac{0 - \exp(\mathbf{u}_o^T \mathbf{v}_c) \cdot \exp(\mathbf{u}_w^T \mathbf{v}_c) \mathbf{v}_c}{[\sum_{k} \exp(\mathbf{u}_k^T \mathbf{v}_c)]^2}$$

$$= -\frac{\exp(\mathbf{u}_o^T \mathbf{v}_c)}{\sum_{k} \exp(\mathbf{u}_k^T \mathbf{v}_c)} \cdot \frac{\exp(\mathbf{u}_w^T \mathbf{v}_c)}{\sum_{k} \exp(\mathbf{u}_k^T \mathbf{v}_c)} \mathbf{v}_c$$

$$= -\hat{y}_o \hat{y}_w \mathbf{v}_c$$

So: $$\frac{\partial J}{\partial \mathbf{u}_w} = -\frac{1}{\hat{y}_o} \cdot (-\hat{y}_o \hat{y}_w \mathbf{v}_c) = \hat{y}_w \mathbf{v}_c$$

Since $y_w = 0$ for $w \neq o$:
$$\boxed{\frac{\partial J}{\partial \mathbf{u}_w} = \mathbf{v}_c(\hat{y}_w - y_w) \text{ for } w \neq o}$$

## Final Answer

For both cases, we can write the unified expression:

$$\boxed{\frac{\partial J}{\partial \mathbf{u}_w} = \mathbf{v}_c(\hat{y}_w - y_w) \text{ for all } w}$$

where:
- When $w = o$: $y_w = y_o = 1$
- When $w \neq o$: $y_w = 0$

Looking at the structure of the matrix **U** and the partial derivatives I computed in the previous part, I need to arrange the column vector derivatives to form the gradient matrix.

Since **U** is a matrix where each column is an outside word vector:
$$\mathbf{U} = [\mathbf{u}_1, \mathbf{u}_2, \ldots, \mathbf{u}_{|\text{Vocab}|}]$$

The partial derivative with respect to **U** will have the same structure, where each column is the partial derivative with respect to the corresponding column vector of **U**.

From the previous part, I found that:
$$\frac{\partial J}{\partial \mathbf{u}_w} = \mathbf{v}_c(\hat{y}_w - y_w) \text{ for all } w$$

Therefore:

$$\boxed{\frac{\partial J(\mathbf{v}_c, o, \mathbf{U})}{\partial \mathbf{U}} = \left[\frac{\partial J(\mathbf{v}_c, o, \mathbf{U})}{\partial \mathbf{u}_1}, \frac{\partial J(\mathbf{v}_c, o, \mathbf{U})}{\partial \mathbf{u}_2}, \ldots, \frac{\partial J(\mathbf{v}_c, o, \mathbf{U})}{\partial \mathbf{u}_{|\text{Vocab}|}}\right]}$$

This can be written more compactly as:

$$\boxed{\frac{\partial J(\mathbf{v}_c, o, \mathbf{U})}{\partial \mathbf{U}} = \mathbf{v}_c(\hat{\mathbf{y}} - \mathbf{y})^T}$$

**Explanation:** The gradient matrix has the same dimensions as **U**. Each column corresponds to the gradient with respect to the corresponding outside word vector. The compact form shows that we can compute this as the outer product of the center vector $\mathbf{v}_c$ with the difference between predicted and true probability distributions $(\hat{\mathbf{y}} - \mathbf{y})$.